In [1]:
import os
import keyring

OPENAI_API_KEY = keyring.get_password('openai', 'key_for_windows')
SERPER_API_KEY = keyring.get_password('serper', 'key_for_windows')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['SERPER_API_KEY'] = SERPER_API_KEY

In [2]:
from crewai import Agent, Task, Crew
from crewai.process import Process

# import CrewAI Tools
from crewai_tools import (
    SerperDevTool,
    WebsiteSearchTool,
    ScrapeWebsiteTool
)

In [4]:
from datetime import datetime
now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
now

'2024-11-27 00:14:31'

In [21]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini', api_key=OPENAI_API_KEY)
llm_writer = ChatOpenAI(model='gpt-4o', api_key=OPENAI_API_KEY)
llm_editor = ChatOpenAI(model='gpt-4o', api_key=OPENAI_API_KEY)

search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()
scrap_tool = ScrapeWebsiteTool()

# define agents

researcher = Agent(
    role='tech trend researcher',
    goal=f"Research and summarize AI tech trends and key issues on the given topic. Today: {now}",
    backstory="You are a professional analyst with a keen eye for tech trends and an AI developer.",
    tools=[search_tool, web_rag_tool],
    verbose=True,
    max_iter=5,
    llm=llm
)

developer = Agent(
    role='AI engineer',
    goal=f'Write examples with the latest code on the given topic. Today: {now}',
    backstory='You are a senior AI developer, proficient in AI development and skilled at providing explanations for beginner developers.',
    tools=[search_tool, web_rag_tool, scrap_tool],
    verbose=True,
    max_iter=5,
    llm=llm
)

# operator = Agent(
#     role='operator',
#     goal='Provide the editor with a sample blog to illustrate the style or tone of the writing.',
#     backstory='You are very familiar with previous blog posts',
#     tools=[web_rag_tool, scrap_tool],
#     verbose=True,
#     llm=llm
# )

writer = Agent(
    role='writer',
    goal=f"""Based on the summary from a tech trends researcher, write an engaging and user-friendly blog post on an AI topic. 
    Use example code written by an AI engineer to explain the content of the post, and refer to a blog post suggested by the operator to create a similar style and format.
    Today: {now}
    """,
    backstory="You are professional tech blog editor with easy and interesting writing skills.",
    verbose=True,
    tools=[scrap_tool],
    allow_delegation=False,
    llm=llm_editor
)

editor = Agent(
    role="editor",
    goal=f"""You are an organizer and professional editor working with researcher, technician, and writer.
    Verify if the blog post aligns with the topic and validate the accuracy of its content.
    If not, ask researcher, developer and writer to try again. Today: {now}
    """,
    backstory="You are a very professional about AI Technology.",
    verbose=True,
    allow_delegation=False,
    llm=llm_editor,
)

In [22]:
# Define tasks

def define_task(subject):
    
    research = Task(
        description=f"""Research and summarize the definition, background, very recent trends, and pros and cons of {subject}, including references.""",
        expected_output=f'Summary about {subject}',
        agent=researcher,
    )
    
    develop = Task(
        description=f'Write example code that effectively explains the summary about {subject} provided by the researcher.',
        expected_output=f'Example codes about the given summary about {subject}',
        agent=developer
    )
    
    # operate = Task(
    #     description="Give the example for the style and tone of posts. https://anonx.ai/2024/09/19/understanding-sentiment-analysis-in-finance/",
    #     expected_output='blog posts',
    #     agent=operator
    # )
    
    write = Task(
        description=f"""Based on the researcher's summary and the developer's code, 
        write a blog post on {subject}, including an introduction, overview, key technical features, detailed content, latest trends with example code, and a conclusion. 
        Refer to the blog style from https://anonx.ai/2024/09/19/understanding-sentiment-analysis-in-finance.
        Include references from researcher and developer.""",
        agent=writer,
        expected_output=f"A blog post in Markdown format with 6 or more paragraphs introducing the latest tech news in an engaging and humorous tone about {subject}",
        output_file=r'./output/{subject}.md'
    )
    
    return research, develop, write


In [23]:
# define a crew
research, develop, write = define_task("Music Generation Tools intorduction, comparison, and conclusion")

crew = Crew(
    agents=[researcher, developer, writer],
    tasks=[research, develop, write],
    verbose=True,
    process=Process.hierarchical,
    manager_agent=editor,
)

2024-11-27 00:24:43,291 - 2000 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [24]:
# execute tasks
result = crew.kickoff()

# Agent: editor
## Task: Research and summarize the definition, background, very recent trends, and pros and cons of Music Generation Tools intorduction, comparison, and conclusion, including references.
# Agent: tech trend researcher
## Task: Research and provide a summary of the very recent trends in Music Generation Tools, focusing on new developments, technologies, and applications in the field.


# Agent: tech trend researcher
## Thought: I need to gather the latest information on trends in music generation tools, focusing on new developments, technologies, and applications in the field. I'll start by searching the internet for recent articles and updates on this topic.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"recent trends in music generation tools 2024\"}"
## Tool Output: 

Search results: Title: 10 AI Music Generators for Creators in 2024 | DigitalOcean
Link: https://www.digitalocean.com/resources/articles/ai-music-generators
Snippet: Unique soun

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.14it/s]




# Agent: tech trend researcher
## Thought: Thought: I have gathered several relevant articles that discuss recent trends and developments in music generation tools. I will review these articles to extract key insights and summarize them for the report.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"music generation tools trends 2024\", \"website\": \"https://www.digitalocean.com\"}"
## Tool Output: 
Relevant Content:
Top 10 Generative AI Trends in 2024 | Master of Code Global Skip to content Generative AI Trends: Transforming Business and Shaping Future Updated September 23, 2024 Illia Vietrov Former Digital Advertising Specialist Generative AI is an extremely versatile tool that has found its application in various fields. Therefore, it has the potential to become “general-purpose technology.” Moreover, researchers hope to build artificial general intelligence (AGI). Thus, it can turn into a machine that can perform any task that a human can. Let’s

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.82it/s]




# Agent: AI engineer
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"Google MusicLM example code music generation\", \"website\": \"https://magenta.tensorflow.org/\"}"
## Tool Output: 
Relevant Content:
Magenta Get Started Studio DDSP-VST Demos Blog Research Talks Community What is Magenta? An open source research project exploring the role of machine learning as a tool in the creative process. What's new? Magenta Studio 2.0 Magenta Studio has been upgraded to more seamlessly integrate with Ableton Live. It is a collection of music creativity tools built on Magenta’s open source models, using cutting-edge machine learning techniques for music generation. Read the blog post. The 2023 I/O Preshow – Composed by Dan Deacon (with some help from MusicLM) A look into Dan Deacon's creative process for the 2023 Google I/O preshow. Read the blog post. The Wordcraft Writers Workshop: Creative Co-Writing with AI We invited 13 professional writers to explore the 

In [25]:
from IPython.display import display, Markdown

display(Markdown(result.raw))

```markdown
# AI-Driven Music Generation: A Symphony of Bytes and Beats 🎶

Welcome to the dawn of a new musical era, where artificial intelligence meets creativity, and algorithms compose symphonies! In today's blog, we're diving into the world of AI-driven music generation. Whether you're a seasoned composer looking for inspiration or a tech enthusiast curious about the future of music, there's something here for you.

## The Overture: Understanding AI Music Generation

Picture this: you're lounging in your living room, and you casually instruct your AI assistant, "Play me a new song with a jazz flair and a sprinkle of Beethoven." Moments later, your room fills with a melody that’s both fresh and timeless. This isn't a scene from a sci-fi movie—it's the reality brought to us by AI music tools like OpenAI's MuseNet and Google's MusicLM.

These platforms leverage the power of deep learning to create stunning musical pieces. MuseNet can juggle ten different instruments and blend styles from classical to pop, while MusicLM crafts high-fidelity tracks from simple text descriptions. It’s like having a digital Mozart at your fingertips!

## A Crescendo of Features: MuseNet and MusicLM

### OpenAI's MuseNet: The Maestro Neural Network

MuseNet is a deep neural network that can generate four-minute musical compositions using up to ten instruments. The magic lies in its ability to blend styles seamlessly, creating unique compositions that feel both familiar and innovative.

**Example Code:**

```python
import torch
from open_musenet import generate_music  # hypothetical import

style = "classical"
length = 120  # Length of the composition in seconds

music = generate_music(style=style, length=length)
music.save("generated_music.mid")
```

While OpenAI has discontinued direct API access, fans of MuseNet can still explore open-source alternatives inspired by its architecture.

### Google's MusicLM: Text-to-Music Alchemy

Then there’s MusicLM, Google's brainchild, which can transform descriptive text into music. Imagine typing, "Create a lively salsa piece with a dash of tropical flair," and voilà—a new tune is born.

**Example Code:**

```python
import requests

text_prompt = "A peaceful morning with birds chirping and gentle piano music."
url = "https://api.musiclm.com/generate"

response = requests.post(url, json={"text": text_prompt})

if response.status_code == 200:
    music_data = response.content
    with open("generated_music.mp3", "wb") as music_file:
        music_file.write(music_data)
else:
    print("Error generating music:", response.status_code)
```

MusicLM is still a proprietary tool, so while this code illustrates its potential use, keep dreaming about that API access!

## The Rondo: Text-to-Music Innovations

Meta’s MusicGen is another player in this field, providing a bridge between text descriptions and musical creations. It's like telling a bedtime story to your computer and getting a lullaby in return.

**Example Code:**

```python
from musicgen import MusicGen  # hypothetical import

musicgen = MusicGen()
description = "An upbeat electronic track with a playful melody."

generated_song = musicgen.generate(description)
generated_song.save("upbeat_electronic_track.mp3")
```

## Coda: The Future of Music Creation

As AI technology continues to evolve, the potential for music generation tools is boundless. These platforms not only enhance creative expression but also democratize music production, allowing anyone with a computer to compose like a pro. Whether you're a tech-savvy musician or a curious listener, the future sounds promising.

In the spirit of musical exploration, why not experiment with these tools yourself? Dive into the world of AI music generation and let your creativity resonate with the rhythm of innovation. Who knows—your next favorite song might just be a few algorithms away!

So, tune in, code on, and let the symphony of AI-generated music play!

---

This post was inspired by the engaging style of [anonx.ai's Sentiment Analysis in Finance](https://anonx.ai/2024/09/19/understanding-sentiment-analysis-in-finance), blending technical insight with a touch of humor. We hope you enjoyed this melodic journey through AI music generation!
```